In [27]:
import pandas as pd
import numpy as np
import os

def championship_impact(laps_path, p_max=26):
    standing = pd.read_csv(f"{laps_path}/championship_drivers.csv")
    df = standing.copy()
    df["position_start"] = df["position_start"].fillna(10)
    session_key = df["session_key"].max()

    df["w"] = 1 / df["position_start"]
    df["points_delta"] = (df["points_current"] - df["points_start"]).abs()
    df["weighted_delta"] = df["points_delta"] * df["w"]

    pri = df["weighted_delta"].sum() / (df["w"].sum() * p_max)

    grid_size = len(df)
    sci = (
        (df["position_current"] - df["position_start"])
        .abs()
        .mean()
        / (grid_size - 1)
    )

    impact = 0.5 * pri + 0.5 * sci

    return {
        "session_key": session_key,
        "PRI": pri,
        "SCI": sci,
        "championship_impact": impact
    }

In [24]:
championship_impact("data/raw/2025/AUS - Melbourne")

{'session_key': np.int64(9693),
 'PRI': np.float64(0.1942307692307692),
 'SCI': np.float64(0.5026315789473684),
 'championship_impact': np.float64(0.3484311740890688)}

In [28]:
import os
def collect_championship_impact(base_path) :
    results = []

    # Проходим по всем подкаталогам
    for gp_folder in os.listdir(base_path):
        race_path = os.path.join(base_path, gp_folder)
        if not os.path.isdir(race_path):
            continue

        try:
            impact = championship_impact(race_path)
            outcome_dict = {"session_key": impact.get("session_key", gp_folder),
                            "PRI": impact.get("PRI", None),
                            "SCI": impact.get("SCI", None),
                            "championship_impact_score": impact.get("championship_impact", None)
                            }

            


            results.append(outcome_dict)

        except Exception as e:
            print(f"Ошибка при обработке {gp_folder}: {e}")
            continue

    # Собираем в DataFrame
    df_metrics = pd.DataFrame(results)
    df_metrics.to_csv("data/processed/2025/championship_impact.csv", index=False)
    return df_metrics

In [29]:
collect_championship_impact("data/raw/2025")

,session_key,PRI,SCI,championship_impact_score
0,10022,0.414332,0.015789,0.215061
1,10033,0.495190,0.015789,0.255490
2,9987,0.410651,0.023810,0.217230
3,9955,0.414889,0.033333,0.224111
4,10014,0.386236,0.042105,0.214171
5,10006,0.446149,0.057895,0.252022
6,9912,0.421237,0.019048,0.220143
7,9877,0.372045,0.028571,0.200308
8,9858,0.425856,0.014286,0.220071
9,9888,0.372028,0.014286,0.193157
